# Prepare NLPText

In [1]:
from pprint import pprint
from nlptext.base import BasicObject

CORPUSPath = 'corpus/boson/'

Corpus2GroupMethod = '.txt'

Group2TextMethod   = 'line'

Text2SentMethod  = 're'

Sent2TokenMethod = 'iter'
TOKENLevel = 'char'

min_token_freq = 1

use_hyper = ['pos']

anno = 'anno_embed_in_text'
anno_keywords = {}

BasicObject.INIT(CORPUSPath, 
                 Corpus2GroupMethod, 
                 Group2TextMethod, 
                 Text2SentMethod, 
                 Sent2TokenMethod, TOKENLevel, min_token_freq = min_token_freq,
                 use_hyper = use_hyper, 
                 anno = anno, anno_keywords = anno_keywords)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


'File'
corpus/boson/bosonNER.txt


Loading model cost 0.660 seconds.
Prefix dict has been built succesfully.


Total Num of All    Tokens 533491
Total Num of Unique Tokens 3825
CORPUS	it is Dumped into file: data/boson/char/Pyramid/CORPUS.p
CORPUS	the length of it is   : 1
GROUP	it is Dumped into file: data/boson/char/Pyramid/GROUP.p
GROUP	the length of it is   : 1
TEXT	it is Dumped into file: data/boson/char/Pyramid/TEXT.p
TEXT	the length of it is   : 1961
SENT	it is Dumped into file: data/boson/char/Pyramid/SENT.p
SENT	the length of it is   : 10214
TOKEN	it is Dumped into file: data/boson/char/Pyramid/TOKEN.p
TOKEN	the length of it is   : 533491
**************************************** 

pos-bioes	is Dumped into file: data/boson/char/Vocab/pos-bioes.voc
pos-bioes	the length of it is   : 229
		Write to: data/boson/char/Vocab/pos-bioes.tsv
annoE-bioes	is Dumped into file: data/boson/char/Vocab/annoE-bioes.voc
annoE-bioes	the length of it is   : 25
		Write to: data/boson/char/Vocab/annoE-bioes.tsv
token   	is Dumped into file: data/boson/char/Vocab/token.voc
token   	the length of it is   : 3825

In [2]:
CHANNEL_SETTINGS_TEMPLATE = {
    # CTX_IND
    'token':   {'use': True,'Max_Ngram': 1,}, # always the char-level token
    'basic':   {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'medical': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'radical': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'subcomp': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'stroke':  {'use': False,'Max_Ngram': 1, 'end_grain': False},
    # CTX_DEP
    'pos':     {'use': False, 'tagScheme': 'BIOE',},
    # ANNO
    'annoE':   {'use': False, 'tagScheme': 'BIOE',},
}

BasicObject.BUILD_GV_LKP(CHANNEL_SETTINGS_TEMPLATE=CHANNEL_SETTINGS_TEMPLATE)


Deal with the Channel: token
Current Channel is        	 token
Current Channel Max_Ngram 	 1


In [3]:

Target_Field = [
    'annoE', {'tagScheme': 'BIOES'}
]



def get_target_field_info(nlptext, Target_Field, fldembed, useStartEnd = False):
    field, para = Target_Field
    # nlptext is important here.
    # channel_anno = 'annoE'
    target_para = {}
    
    if field == 'token':
        # use start and end only works for token
        special_num = 3 if useStartEnd else 1 
        # from fldembed
        Weights = fldembed.weights
        assert 'token' in Weights
        wv = Weights['token']
        GU = wv.GU
        idx2grain = ['</pad>'] + GU[0]
        grain_num = len(idx2grain)
        grain2idx = dict(zip(idx2grain, list(range(grain_num)) ))
        GU = idx2grain, grain2idx
        target_para['GU']  = GU # </pad> in GU, no </unk> in GU
        labels = [] 
        tag_size = len(idx2grain) + special_num # </pad> in GU, and </unk>, </start>, </end> not in GU

    else: 
        # generally, we won't adding start and end into CRF model.
        # even the features are with start and end, it will should be removed before feed into crf
        # it can also be proved that the start and end vectors are useless in sequential labeling.
        GU = nlptext.getGrainVocab(field, **para)
        idx2tag = ['</pad>'] + GU[0]
        tag2idx = dict(zip(range(len(idx2tag)), idx2tag))
        GU = idx2tag, tag2idx
        # </pad> is in GU, no </unk> or other special tags in GU
        tag_size = len(idx2tag)       
        target_para['GU'] = GU
        
        TRANS = nlptext.getTrans(field, **para)
        TRANS = {k: v+1 for k, v in TRANS.items()}
        labels = list(set([i.split('-')[0] for i in idx2tag if '-' in i]))
        labels.sort()
        target_para['TRANS'] = TRANS
        
    # target_para['labels'] = labels
    # target_para['tag_size'] = tag_size
    
    TARGET_FIELD = [field, target_para, labels, tag_size]
    return TARGET_FIELD


nlptext = BasicObject
get_target_field_info(nlptext, Target_Field, None, useStartEnd = False)

['annoE',
 {'GU': (['</pad>',
    'O',
    'person_name-B',
    'person_name-I',
    'person_name-E',
    'person_name-S',
    'product_name-B',
    'product_name-I',
    'product_name-E',
    'product_name-S',
    'org_name-B',
    'org_name-I',
    'org_name-E',
    'org_name-S',
    'location-B',
    'location-I',
    'location-E',
    'location-S',
    'time-B',
    'time-I',
    'time-E',
    'time-S',
    'company_name-B',
    'company_name-I',
    'company_name-E',
    'company_name-S'],
   {0: '</pad>',
    1: 'O',
    2: 'person_name-B',
    3: 'person_name-I',
    4: 'person_name-E',
    5: 'person_name-S',
    6: 'product_name-B',
    7: 'product_name-I',
    8: 'product_name-E',
    9: 'product_name-S',
    10: 'org_name-B',
    11: 'org_name-I',
    12: 'org_name-E',
    13: 'org_name-S',
    14: 'location-B',
    15: 'location-I',
    16: 'location-E',
    17: 'location-S',
    18: 'time-B',
    19: 'time-I',
    20: 'time-E',
    21: 'time-S',
    22: 'company_name-B',
 

# Featurize a Sentence

In [4]:
from nlptext.sentence import Sentence
sent = Sentence(12)
print(sent.Idx)
print(sent)
print(sent.sentence)
Channel_Settings = BasicObject.CHANNEL_SETTINGS
print(Channel_Settings)

12
<st 12 (tokenNum: 155) >
H M C 证 券 分 析 师 格 雷 格 ・ 罗 ( G r e g R o h ) 指 出 : “ 与 其 他 二 线 智 能 手 机 品 牌 相 比 , L G 手 机 的 质 量 有 了 一 定 的 提 升 , 但 在 品 牌 知 名 度 及 出 货 量 方 面 , L G 与 三 星 、 苹 果 等 手 机 巨 头 仍 有 不 小 的 差 距 。 ” L G 手 机 在 工 业 设 计 及 产 品 本 身 以 及 渠 道 方 面 都 与 三 星 存 在 着 一 定 的 差 距 , 从 软 件 到 硬 件 , 产 品 的 细 节 化 始 终 没 有 三 星 完 善 , 超 越 之 路 任 重 道 远 。
{'token': {'Max_Ngram': 1}}


## String Features

In [5]:
import pandas as pd

def get_sent_strfeats(sent, Channel_Settings, train = True):
    '''
        sent is a nlptex.sentence object
        return a pandas dataframe
    '''
    try:
        df = sent.feats
        columns = df.columns
        new_columns = [i for i in columns if i.split('_')[0] in Channel_Settings]
        if not train:
            new_columns = [i for i in new_columns if 'anno' not in i]
        Feats = df.loc[new_columns]
        return Feats
    except:
        features = {}
        # stroke 12 and subcomp 6 are fixed internally.
        for ch, cs in Channel_Settings.items():
            if 'anno' in ch and not train:
                continue
            feature = sent.get_grain_str(ch)
            # this will cost a lot of time
            if ch == 'stroke':
                max_leng = 12
                feature2 = []
                for token_feat in feature:
                    if len(token_feat) <= max_leng:
                        feature2.append(token_feat + ['</'] * (max_leng - len(token_feat))) 
                    else:
                        feature2.append(token_feat[:max_leng])
                feature = feature2
            elif ch == 'subcomp':
                max_leng = 6
                feature2 = []
                for token_feat in feature:
                    if len(token_feat) <= max_leng:
                        feature2.append(token_feat + ['</'] * (max_leng - len(token_feat)))
                    else:
                        feature2.append(token_feat[:max_leng])
                feature = feature2 
            features[ch] = feature
            # print(feature)
        L = []
        for ch, feat in features.items():
            df = pd.DataFrame(feat)
            df.columns = [ch + '_' + str(i) for i in range(df.shape[1])]
            L.append(df)
        
        Feats = pd.concat(L, axis = 1)
        return Feats

In [6]:
# from crfpp.crftools import get_sent_strfeats
get_sent_strfeats(sent, Channel_Settings) # this need some attention

,token_0
0,H
1,M
2,C
3,证
4,券
5,分
6,析
7,师
8,格
9,雷


## Vector Features (TODO)

In [7]:
# from fieldembed import FieldEmbedding
# fieldembed = FieldEmbedding.load('embeddings/fieldembed/Ch_wiki_char_subcomp_model')
# fieldembed.Field_Settings

In [8]:
# wv_subcomp = fieldembed.weights['subcomp']# .GU
# wv_subcomp

In [9]:
# wv_subcomp.derivative_wv.GU

In [10]:
def get_sent_vecfeats(sent, fieldembed, train = True):
    features = {}
    Channel_Settings = fieldembed.Field_Settings
    token_strs = [i[0] for i in sent.get_grain_str('token')]
    # print(token_strs)
    features['origin'] = token_strs
    for ch, cs in Channel_Settings.items():
        if 'anno' in ch and not train:
            continue
        derivative_wv = fieldembed.weights[ch].derivative_wv
        TU = derivative_wv.GU # LGU in derivative wv is LTU
        # this code is verbose
        token_idxes = [TU[1].get(token_str, 0) for token_str in token_strs] # 0 is not unk, to fix it in the future
        # token_idxes = [i[0] for i in token_idxes]
        # print(token_idxes)
        feature = derivative_wv.vectors[token_idxes]
        features[ch] = feature
        # print(feature)
    L = []
    for ch, feat in features.items():
        df = pd.DataFrame(feat)
        df.columns = [ch + '_' + str(i) for i in range(df.shape[1])]
        L.append(df)

    Feats = pd.concat(L, axis = 1)
    return Feats

In [11]:
# # from crfpp.crftools import get_sent_vecfeats

# get_sent_vecfeats(sent, fieldembed)

# Featurize all Sentences

In [12]:
BasicObject

nlptext.base.BasicObject

In [13]:
import os
import pickle


def featurize_nlptext_sentences(BasicObject, feat_type = 'str', fieldembed = None):
    Total_Settings = {
        'token': {'Max_Ngram': 1},
        'char': {'Max_Ngram': 1, 'end_grain': False},
        'basic': {'Max_Ngram': 1, 'end_grain': False},
        'radical': {'Max_Ngram': 1, 'end_grain': False},
        'subcomp': {'Max_Ngram': 1, 'end_grain': False},
        'stroke': {'Max_Ngram': 1, 'end_grain': False},
        'pos': {'Max_Ngram': 1, 'end_grain': False,   'tagScheme': 'BIOE'},
        'annoE': {'Max_Ngram': 1, 'end_grain': False, 'tagScheme': 'BIOE'}
    }

    # prepare path to save the features.
    if 'str' in feat_type.lower():
        sentence_path = BasicObject.Data_Dir + '/' + 'Pyramid/_Feat_SENT_Str.crfpp'
        get_sent_feats = get_sent_strfeats
    elif 'vec' in feat_type.lower():
        sentence_path = BasicObject.Data_Dir + '/' + 'Pyramid/_Feat_SENT_Vec.crfpp'
        get_sent_feats = get_sent_vecfeats
    else:
        print('Error! Currently there is no such a feature type')
        
    # if there is 
    if os.path.isfile(sentence_path):

        with open(sentence_path, 'rb') as handle:
            sents = pickle.load(handle)
        print('Load Featurized Sentences from:')
        print('\t', sentence_path)
        return sents
    else:
        from nlptext.sentence import Sentence
        # corpus = Corpus() # this costs time
        sents = []
        if fieldembed:
            Total_Settings = fieldembed
        for sentidx in range(BasicObject.SENT['length']):
            sent = Sentence(sentidx)
            sent.feats = get_sent_feats(sent, Total_Settings)
            sents.append(sent)
        with open(sentence_path, 'wb') as handle:
            pickle.dump(sents, handle)
        print('Save Featurized Sentences to:')
        print('\t', sentence_path)
        return sents

## Featurize all Sentences with String

In [14]:
sents = featurize_nlptext_sentences(BasicObject, feat_type = 'str')

		Build GrainUnique for channel: pos-bio
pos 1 False BIO
		Write to: data/boson/char/Vocab/pos-bio.voc
		Write to: data/boson/char/Vocab/pos-bio.tsv
		Build GrainUnique for channel: annoE-bio
annoE 1 False BIO
		Write to: data/boson/char/Vocab/annoE-bio.voc
		Write to: data/boson/char/Vocab/annoE-bio.tsv
Save Featurized Sentences to:
	 data/boson/char/Pyramid/_Feat_SENT_Str.crfpp


## Featurize all Sentences with Vector (TODO)

In [15]:
# from fieldembed import FieldEmbedding
# fieldembed = FieldEmbedding.load('embeddings/fieldembed/Ch_wiki_char_subcomp_model')
# fieldembed.Field_Settings

# sents = featurize_nlptext_sentences(BasicObject, feat_type = 'vec', fieldembed=fieldembed)

# Generate Template

The `template` is used to convert `input feats` to `derivative feats`.

In [18]:
import pandas as pd
import numpy as np

# individaul

individual_para = {
    1: 0,
}

def generate_template(input_feats_num = 5, individual_para = individual_para, path = None):
    '''
     this still needs more consideration
    '''
    if not path:
        path = '_template'
        
    L = ['# Unigram\n\n']
    fld_idx = 0
    for feat_idx in range(input_feats_num):
        for gram_num, window_size in individual_para.items():
            if gram_num == 1:
                for token_i in range(-window_size, window_size + 1):
                    L.append('U{}:%x[{},{}]\n'.format(str(fld_idx), str(token_i), str(feat_idx)))
                    fld_idx = fld_idx + 1
            if gram_num == 2 and feat_idx <= 5:
                for token_i in range(-window_size, window_size + 1):
                    L.append('U{}:%x[{},{}]/%x[{},{}]\n'.format(str(fld_idx), 
                                                                str(token_i), str(feat_idx), 
                                                                str(token_i + 1), str(feat_idx)))
                    fld_idx = fld_idx + 1
                    
            if gram_num == 3 and feat_idx <= 5:
                for token_i in range(-window_size, window_size + 1):
                    L.append('U{}:%x[{},{}]/%x[{},{}]/%x[{},{}]\n'.format(str(fld_idx), 
                                                                          str(token_i - 1), str(feat_idx), 
                                                                          str(token_i), str(feat_idx), 
                                                                          str(token_i + 1), str(feat_idx)))
                    fld_idx = fld_idx + 1
    
    
    L.append('\n\n# Bigram\nB')
    
    with open(path, 'w') as f:
        f.write(''.join(L))
        
    return ''.join(L)

print(generate_template(input_feats_num = 20))

# Unigram

U0:%x[0,0]
U1:%x[0,1]
U2:%x[0,2]
U3:%x[0,3]
U4:%x[0,4]
U5:%x[0,5]
U6:%x[0,6]
U7:%x[0,7]
U8:%x[0,8]
U9:%x[0,9]
U10:%x[0,10]
U11:%x[0,11]
U12:%x[0,12]
U13:%x[0,13]
U14:%x[0,14]
U15:%x[0,15]
U16:%x[0,16]
U17:%x[0,17]
U18:%x[0,18]
U19:%x[0,19]


# Bigram
B


#  Load Data

In [27]:
sents

[<st 0 (tokenNum: 40) >,
 <st 1 (tokenNum: 23) >,
 <st 2 (tokenNum: 75) >,
 <st 3 (tokenNum: 28) >,
 <st 4 (tokenNum: 57) >,
 <st 5 (tokenNum: 17) >,
 <st 6 (tokenNum: 41) >,
 <st 7 (tokenNum: 112) >,
 <st 8 (tokenNum: 65) >,
 <st 9 (tokenNum: 64) >,
 <st 10 (tokenNum: 66) >,
 <st 11 (tokenNum: 90) >,
 <st 12 (tokenNum: 155) >,
 <st 13 (tokenNum: 113) >,
 <st 14 (tokenNum: 63) >,
 <st 15 (tokenNum: 59) >,
 <st 16 (tokenNum: 68) >,
 <st 17 (tokenNum: 28) >,
 <st 18 (tokenNum: 38) >,
 <st 19 (tokenNum: 13) >,
 <st 20 (tokenNum: 146) >,
 <st 21 (tokenNum: 39) >,
 <st 22 (tokenNum: 30) >,
 <st 23 (tokenNum: 20) >,
 <st 24 (tokenNum: 55) >,
 <st 25 (tokenNum: 29) >,
 <st 26 (tokenNum: 15) >,
 <st 27 (tokenNum: 11) >,
 <st 28 (tokenNum: 60) >,
 <st 29 (tokenNum: 43) >,
 <st 30 (tokenNum: 117) >,
 <st 31 (tokenNum: 25) >,
 <st 32 (tokenNum: 125) >,
 <st 33 (tokenNum: 28) >,
 <st 34 (tokenNum: 45) >,
 <st 35 (tokenNum: 88) >,
 <st 36 (tokenNum: 21) >,
 <st 37 (tokenNum: 145) >,
 <st 38 (tokenN

In [17]:
print(len(sents))

10214


In [24]:
from crfpp.loaddata import get_train_test_valid

total_sent_num = nlptext.SENT['length']
train_sent_idx, test_sent_idx, valid_sent_idx = get_train_test_valid(total_sent_num, seed = 10)

The num of train sentences: 6128
The num of test  sentences: 2043
The num of valid sentences: 2043


In [28]:
train_sents = [sents[i] for i in train_sent_idx]
test_sents  = [sents[i] for i in test_sent_idx]
valid_sents = [sents[i] for i in valid_sent_idx]

# Train Model

In [29]:
from nlptext.utils.channel import getChannelName

def get_model_name(BasicObject, Channel_Settings):
    return BasicObject.Data_Dir.replace('data/', 'model/') + '/' + "_".join([getChannelName(ch, style = 'abbr') for ch in Channel_Settings])
# Channel_Settings
model = BasicObject.Data_Dir.replace('data/', 'model/') + '/' + "_".join([getChannelName(ch, style = 'abbr') for ch in Channel_Settings])
model

'model/boson/char/T'

In [21]:
import os
from datetime import datetime
from crfpp.crftools import crf_learn
import pickle

def crfpp_train(model, trainSents, Channel_Settings, feat_type = 'str', fieldembed = None):
    
    if 'str' in feat_type.lower():
        get_sent_feats = get_sent_strfeats
    elif 'vec' in feat_type.lower():
        get_sent_feats = get_sent_vecfeats
        
    tmp_dir = model.replace('model', '_tmp')
    
    if not os.path.exists(tmp_dir):
        os.makedirs(tmp_dir)
        
    if not os.path.exists(model):
        os.makedirs(model)
        
    model_path = model + '/model'
    para_path  = model + '/para.p'
    template_path = model + '/template'
    feats_data_path = tmp_dir + '/feats.txt'
    with open(para_path, 'wb') as f:
        pickle.dump(Channel_Settings, f)
    
    DFtrain = []
    if fieldembed:
        Channel_Settings = fieldembed
    for idx, sent in enumerate(trainSents):
        if idx % 500 == 0:
            print(datetime.now(), idx)
        df = get_sent_feats(sent, Channel_Settings)
        df.loc[len(df)] = np.NaN     ## Trick Here
        DFtrain.append(df)           ## Trick Here
    DFtrain = pd.concat(DFtrain).reset_index(drop=True)
    DFtrain.to_csv(feats_data_path, sep = '\t', encoding = 'utf=8', header = False, index = False )
    
    generate_template(input_feats_num = DFtrain.shape[1] - 1, path = template_path) 
    crf_learn(feats_data_path, model_path, template_path  = template_path)
    
    return None

In [30]:
Channel_Settings

{'token': {'Max_Ngram': 1}}

In [31]:
crfpp_train(model, train_sents, Channel_Settings, feat_type = 'str', fieldembed = None)

2019-07-31 16:17:17.578930 0
2019-07-31 16:17:18.615698 500
2019-07-31 16:17:19.651774 1000
2019-07-31 16:17:20.699888 1500
2019-07-31 16:17:21.688091 2000
2019-07-31 16:17:22.693033 2500
2019-07-31 16:17:23.909671 3000
2019-07-31 16:17:24.913635 3500
2019-07-31 16:17:25.890951 4000
2019-07-31 16:17:27.001635 4500
2019-07-31 16:17:27.971038 5000
2019-07-31 16:17:29.061359 5500
2019-07-31 16:17:30.063687 6000
CRF++: Yet Another CRF Tool Kit
Copyright (C) 2005-2013 Taku Kudo, All rights reserved.

reading training data: 
Done!11.93 s

Number of sentences: 1
Number of features:  12524521
Number of thread(s): 8
Freq:                2
eta:                 0.00010
C:                   5.00000
shrinking size:      20



# Tagger a Sentence

In [24]:
sent = testSents[21]
print(sent)
print(sent.sentence)
print(model)

<st 103 (tokenNum: 312) >
一 到 节 假 日 , 不 论 是 旅 游 景 点 还 是 餐 厅 、 K T V 等 地 都 是 人 满 为 患 , 本 打 算 团 购 一 个 美 食 套 餐 , 在 假 期 约 上 三 五 好 友 共 享 一 番 , 然 而 多 次 预 约 都 已 满 , 多 位 网 友 均 向 本 平 台 反 映 团 购 的 套 餐 由 于 遇 上 长 假 消 费 高 峰 因 此 眼 睁 睁 看 着 过 期 , 消 费 不 成 , 退 款 也 无 门 。 ( 相 关 案 例 详 见 : 订 不 到 餐 是 否 可 退 窝 窝 网 、 澳 门 豆 捞 说 法 不 一 1 0 0 e c . c n / d e t a i l - - 6 0 0 3 0 3 5 . h t m l ) 经 常 网 购 的 朋 友 一 定 发 现 了 促 销 期 间 的 发 货 时 间 是 平 常 的 3 - 4 倍 , 由 于 促 销 期 间 , 商 家 订 单 数 量 大 , 往 往 就 发 货 不 及 时 , 因 此 , 网 友 下 单 后 本 来 两 三 天 能 收 到 的 商 品 可 能 十 天 也 未 必 能 收 到 。 ( 相 关 案 例 详 见 : 搜 狐 爱 家 团 付 款 订 单 不 发 货 显 示 状 态 未 支 付 1 0 0 e c . c n / d e t a i l - - 6 0 0 3 0 3 8 . h t m l )
model/boson/char/T_E-bio


In [25]:
from crfpp.crftools import get_sent_strfeats, crf_test
from crfpp.evals import read_target_seq, extractSET

def tagger(model, sent, Channel_Settings = None):
    '''
        basically from crf_test
        sent: a sentence, could be without annotation
    '''
    if not Channel_Settings:
        with open(model + '/para.p', 'rb') as f:
            Channel_Settings = pickle.load(f)
    # 1. get sentence feats
    # hopefully, the model_config is included in model_path
    feats_data_path   = '_tmp/_tagger_feats.txt'
    results_data_path = '_tmp/_tagger_results.txt'
    model_path = model + '/model'

    # get Channel_Settings
    # get use sent strfeats or sent vecfeats settings
    df = get_sent_strfeats(sent, Channel_Settings, train = False)
    df.to_csv(feats_data_path, sep = '\t', encoding = 'utf=8', header = False, index = False )
    # 2. save the sentence feats to a file
    
    # 3. tag a sentence
    crf_test(feats_data_path, model_path, results_data_path)

    # 4. read and parse the result to pred_SSET
    # get a tag_seq
    # list of tuple (score, result)
    tag_seq  = read_target_seq(results_data_path)
    pred_SET = extractSET(tag_seq)
    return pred_SET

tagger(model, sent) 

[(0, 5, 'time'),
 (18, 21, 'company_name'),
 (129, 132, 'product_name'),
 (133, 135, 'location'),
 (263, 268, 'product_name'),
 (309, 311, 'person_name')]

# Test Model

In [26]:
print(labels)
print(model)

['company_name', 'location', 'org_name', 'person_name', 'product_name', 'time']
model/boson/char/T_E-bio


In [27]:
import pandas as pd
import numpy as np
from tabulate import tabulate

###################################################  from IT to SET 

def read_target_seq(result_path):
    result = pd.read_csv(result_path, sep = '\t', header = None, skip_blank_lines=False)
    # get the last column of the result
    return result.iloc[:,-1].dropna().values


def extractSET(tag_seq, exist_SE = False):
    '''
        SET: start, end, tag
        tag_seq: the hyper field sequence for this sentence

    '''
    if exist_SE:
        tag_seq = tag_seq[1:-1]

    IT = list(zip(range(len(tag_seq)), tag_seq))
    taggedIT = [it for it in IT if it[1]!= 'O']
    
    startIdx = [idx for idx in range(len(taggedIT)) if taggedIT[idx][1][-2:] == '-B' or taggedIT[idx][1][-2:] == '-S']
    startIdx.append(len(taggedIT))

    entitiesList = []
    for i in range(len(startIdx)-1):
        entityAtom = taggedIT[startIdx[i]: startIdx[i+1]]
        # string = ''.join([cit[0] for cit in entityAtom])
        start, end = entityAtom[0][0], entityAtom[-1][0] + 1
        tag = entityAtom[0][1].split('-')[0]
        entitiesList.append((start, end, tag))
    return entitiesList


def get_sent_annoSET(sent, channel = 'annoE', tagScheme = 'BIO'):
    anno_seq = [i[0] for i in sent.get_grain_str(channel, tagScheme=tagScheme)]
    anno_SET = extractSET(anno_seq)
    return anno_SET 



###################################################  compare pred_SET and anno_SET

def match_anno_pred_result(anno_entities, pred_entities, labels = []):
    '''
        anno_entities, pred_entities of a batch of sentences
    '''
    if type(anno_entities[0]) != list:
        anno_entities = [anno_entities]
        pred_entities = [pred_entities]
        
    name_list = ['E@Anno', 'E@Pred',  'E@Match']
    for eL in labels:
        name_list.extend([eL + suff for suff in ['@Anno', '@Pred', '@Match']])
    
    statistic_result = []
    
    for idx in range(len(pred_entities)):
        pred = set(pred_entities[idx])
        anno = set(anno_entities[idx])
        d = {'E@Pred'  : len(pred),
             'E@Anno'  : len(anno),
             'E@Match' : len(pred.intersection(anno))}
        
        for eL in labels:
            elL = [e for e in pred if eL == e[-1]]
            elA = [e for e in anno if eL == e[-1]]
            elM = set(elA).intersection(set(elL)) ## Union vs Join
            d[eL+'@Pred'] = len(elL)
            d[eL+'@Anno'] = len(elA)
            d[eL+'@Match'] = len(elM)
        
        statistic_result.append(d)
    Result = pd.DataFrame(statistic_result)[name_list]
    return Result


def calculate_F1_Score(Result, labels):
    if len(Result.shape) == 1:
        Result = Result.to_dict()
    else:
        Result = Result.sum().to_dict()
    List = []
    entitiesLabel = labels + ['E']
    # entitiesLabel = ['Sy','Bo', 'Ch', 'Tr', 'Si'] + ['R'] + ['E']
    for eL in entitiesLabel:
        d = dict()
        d['id'] = eL
        for k in Result:
            if eL == k.split('@')[0]:
                d[k.replace(eL + '@','')] = Result[k]
        List.append(d)
    
    # print(List)
    R = pd.DataFrame(List)
    R.set_index('id', inplace = True)
    R.index.name = None
    # print(R)
    R['R'] = R['Match']/R['Anno']
    R['P'] = R['Match']/R['Pred']
    R['F1'] = 2*R['R']*R['P']/(R['R'] + R['P'])
    return R[['Anno', 'Pred', 'Match', 'R', 'P', 'F1']]


################################################### log the errors between pred_SET and anno_SET

def matchPaired(L, A, sent):
    start1, end1, e1 = L
    start2, end2, e2 = A
    d = {}
    sentence = sent.sentence
    if set(range(start1, end1+1)).intersection(range(start2, end2+1)):
        idx = set(range(start1, end1+1)).union(range(start2, end2+1))
        # print()
        d['text_part'] = sent.sentence[min(idx): max(idx) + 1]
        d['start'] = min(idx)
        d['end' ]  = max(idx) 
        d['pred'] = sent.sentence[start1: end1]
        d['pred_en'] = e1
        d['anno'] = sent.sentence[start2: end2]
        d['anno_en'] = e2
        d['sent_idx']= sent.Idx # this is important
        return d
    
def matchUnpaired(unpaired, sent, kind):
    d = {}
    sentence = sent.sentence
    d['start'], d['end' ], e = unpaired
    d['text_part'] = sentence[d['start']: d['end' ]]
    d['sent_idx']= sent.Idx
    if kind == 'P':
        d['pred'], d['pred_en'] = d['text_part'], e
        d['anno'], d['anno_en'] = None, None
    else:
        d['pred'], d['pred_en'] = None, None
        d['anno'], d['anno_en'] = d['text_part'], e
    return d


def logErrors(sent, anno_enetities, pred_entities):
    log = []
    inter = list(set(pred_entities).intersection(set(anno_enetities)))
    only_pred = [ i for i in pred_entities if i not in inter]   
    only_anno = [ i for i in anno_enetities if i not in inter]
    
    pairedP = []
    pairedA = []
    for L in only_pred:
        for A in only_anno:
            d = matchPaired(L, A, sent)
            if d:
                log.append(d)
                pairedP.append(L)
                pairedA.append(A)
                
    for L in [i for i in only_pred if i not in pairedP]:
        log.append(matchUnpaired(L, sent, 'P'))
        
    for A in [i for i in only_anno if i not in pairedA]:
        log.append(matchUnpaired(A, sent, 'A'))
           
    if len(log) == 0:
        return pd.DataFrame()
    cols = ['sent_idx', 'text_part', 'start', 'end', 'anno', 'anno_en', 'pred', 'pred_en']
    return pd.DataFrame(log)[cols].sort_values('start')



In [28]:
# from crfpp.evals import get_sent_annoSET, match_anno_pred_result, calculate_F1_Score, logError

def crfpp_test(model, testSents, Channel_Settings,  labels):
    '''
        sents: a list of sents
        This function could be updated in the future for a better performance.
        But currently, just leave it now.
    '''
    pred_entities = []
    anno_entities = []
    log_list      = []
    # here sents are a batch of sents, not necessary to be all the sents
    # this could be chanage, but it will cause some time, so just ignore it.
    for idx, sent in enumerate(testSents):
        if idx % 200 == 0:
            print(datetime.now(), idx)
        # 200/13s
        pred_SET = tagger(model, sent, Channel_Settings = Channel_Settings)
        pred_entities.append(pred_SET)
        
        anno_SET = get_sent_annoSET(sent)
        anno_entities.append(anno_SET)
        
        error = logErrors(sent, pred_SET, anno_SET)
        log_list.append(error)
    # return anno_entities, pred_entities
    Result = match_anno_pred_result(anno_entities, pred_entities, labels = labels)
    # return Result
    R = calculate_F1_Score(Result, labels)

    LogError = pd.concat(log_list).reset_index(drop = True)
    return R, LogError


R, LogError = crfpp_test(model, testSents, Channel_Settings,  labels)
R

2019-07-22 17:15:21.014586 0
2019-07-22 17:15:52.238445 200


,Anno,Pred,Match,R,P,F1
company_name,400,307,204,0.510000,0.664495,0.577086
location,900,806,596,0.662222,0.739454,0.698710
org_name,561,381,295,0.525847,0.774278,0.626327
person_name,1063,805,731,0.687676,0.908075,0.782655
product_name,768,639,456,0.593750,0.713615,0.648188
time,885,815,663,0.749153,0.813497,0.780000
E,4577,3753,2945,0.643435,0.784706,0.707083


#  Overall Workflow

In [1]:
from pprint import pprint
from nlptext.base import BasicObject

########### ResumeNER ###########
CORPUSPath = 'corpus/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW)

CHANNEL_SETTINGS_TEMPLATE = {
    # CTX_IND
    'token':   {'use': True,'Max_Ngram': 1,}, # always the char-level token
    'basic':   {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'medical': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'radical': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'subcomp': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'stroke':  {'use': False,'Max_Ngram': 1, 'end_grain': False},
    # CTX_DEP
    'pos':     {'use': False, 'tagScheme': 'BIO',},
    # ANNO
    'annoE':   {'use': True, 'tagScheme': 'BIO',},
}

BasicObject.BUILD_GRAIN_UNI_AND_LOOKUP(CHANNEL_SETTINGS_TEMPLATE=CHANNEL_SETTINGS_TEMPLATE)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


'File'
corpus/ResumeNER/test.char.bmes


Loading model cost 0.632 seconds.
Prefix dict has been built succesfully.


corpus/ResumeNER/train.char.bmes
corpus/ResumeNER/dev.char.bmes
Total Num of All    Tokens 149812
The Total Number of Tokens: 149812
Counting the number unique Tokens...          	 2019-06-20 20:19:23.731256
		Done!
Generating Dictionary of Token Unique...	 2019-06-20 20:19:23.777903
		The length of DTU is: 1891 	 2019-06-20 20:19:23.778263
Generating the ORIGTokenIndex...       	 2019-06-20 20:19:23.778312
		The idx of token is: 0 	 2019-06-20 20:19:23.778461
		Done!
Only Keep First 3500000 Tokens.
The coverage rate is: 0.0
Total Num of Unique Tokens 1891
['</pad>', '</start>', '</end>', 'O', 'CONT-B', 'CONT-E', 'CONT-I', 'CONT-S', 'EDU-B', 'EDU-E', 'EDU-I', 'EDU-S', 'LOC-B', 'LOC-E', 'LOC-I', 'LOC-S', 'NAME-B', 'NAME-E', 'NAME-I', 'NAME-S', 'ORG-B', 'ORG-E', 'ORG-I', 'ORG-S', 'PRO-B', 'PRO-E', 'PRO-I', 'PRO-S', 'RACE-B', 'RACE-E', 'RACE-I', 'RACE-S', 'TITLE-B', 'TITLE-E', 'TITLE-I', 'TITLE-S']
CORPUS	it is Dumped into file: data/ResumeNER/char/Token1891/Pyramid/CORPUS.p
CORPUS	the le

In [2]:
from crfpp.crftools import load_anno

anno_field = 'annoE'
Channel_Settings, tagScheme, labels, tags, tag_size = load_anno(BasicObject, anno_field)
print(Channel_Settings)
print(anno_field)
print(tagScheme)
print(labels)
print(tags)
print(tag_size)

{'token': {'Max_Ngram': 1}, 'basic': {'Max_Ngram': 1, 'end_grain': False}, 'annoE': {'tagScheme': 'BIO'}}
annoE
BIO
['CONT', 'EDU', 'LOC', 'NAME', 'ORG', 'PRO', 'RACE', 'TITLE']
['</pad>', '</start>', '</end>', 'CONT-B', 'CONT-I', 'EDU-B', 'EDU-I', 'LOC-B', 'LOC-I', 'NAME-B', 'NAME-I', 'O', 'ORG-B', 'ORG-I', 'PRO-B', 'PRO-I', 'RACE-B', 'RACE-I', 'TITLE-B', 'TITLE-I']
20


In [3]:
from nlptext.utils.channel import getChannelName

def get_model_name(BasicObject, Channel_Settings):
    return BasicObject.TokenNum_Dir.replace('data/', 'model/') + '/' + "_".join([getChannelName(ch, style = 'abbr') for ch in Channel_Settings if 'anno' not in ch])
# Channel_Settings

model = get_model_name(BasicObject, Channel_Settings)
print(model)

model/ResumeNER/char/Token1891/T_b


In [4]:
from nlptext.corpus import Corpus
corpus = Corpus()
sents = corpus.Sentences

print(model)
print(len(sents))
print(Channel_Settings)

cross_num = 4
cross_validation = True

model/ResumeNER/char/Token1891/T_b
4617
{'token': {'Max_Ngram': 1}, 'basic': {'Max_Ngram': 1, 'end_grain': False}, 'annoE': {'tagScheme': 'BIO'}}


In [5]:
from crfpp.train import train

Performance = train(model, sents, Channel_Settings, labels, cross_num, cross_validation = None, seed = 10)

Performance

2019-06-20 20:19:24.283458 0
2019-06-20 20:19:25.604132 500
2019-06-20 20:19:26.992641 1000
2019-06-20 20:19:28.296988 1500
2019-06-20 20:19:29.613115 2000
2019-06-20 20:19:30.881008 2500
2019-06-20 20:19:32.153796 3000
CRF++: Yet Another CRF Tool Kit
Copyright (C) 2005-2013 Taku Kudo, All rights reserved.

reading training data: 100.. 200.. 300.. 400.. 500.. 600.. 700.. 800.. 900.. 1000.. 1100.. 1200.. 1300.. 1400.. 1500.. 1600.. 1700.. 1800.. 1900.. 2000.. 2100.. 2200.. 2300.. 2400.. 2500.. 2600.. 2700.. 2800.. 2900.. 3000.. 3100.. 3200.. 3300.. 3400.. 
Done!1.03 s

Number of sentences: 3463
Number of features:  1254770
Number of thread(s): 8
Freq:                2
eta:                 0.00010
C:                   5.00000
shrinking size:      20
iter=0 terr=0.99796 serr=1.00000 act=1254770 obj=321255.22787 diff=1.00000
iter=1 terr=0.54466 serr=1.00000 act=1254770 obj=205289.10905 diff=0.36098
iter=2 terr=0.37801 serr=1.00000 act=1254770 obj=146378.28020 diff=0.28697
iter=3 terr=0.337

2019-06-20 20:25:34.054235 200
2019-06-20 20:25:41.800321 400
2019-06-20 20:25:49.584570 600
2019-06-20 20:25:57.297878 800
2019-06-20 20:26:04.847467 1000

The Final Performance is:



,Anno,Pred,Match,R,P,F1
CONT,90,88,88,0.977778,1.000000,0.988764
EDU,263,267,250,0.950570,0.936330,0.943396
LOC,9,8,8,0.888889,1.000000,0.941176
NAME,302,295,294,0.973510,0.996610,0.984925
ORG,1421,1411,1307,0.919775,0.926293,0.923023
PRO,80,85,76,0.950000,0.894118,0.921212
RACE,29,27,27,0.931034,1.000000,0.964286
TITLE,1889,1888,1778,0.941239,0.941737,0.941488
E,4083,4069,3828,0.937546,0.940772,0.939156


# Tag Sentence

In [22]:
from crfpp.tagger import tagger
from nlptext.sentence import Sentence

model = 'model/MedPos/char/T_b-n1t1-f1_m-n1t1-f1_r-n1t1-f1_P-bio_E-bio'
sent_str = '（6）不能规律进食，或不能配合饮食干预者'

sent = Sentence(sentence=[i for i in sent_str.replace(' ', '@')])

TOKENLevel = 'word' if 'word' in model else 'char'
sent.TOKEN['TOKENLevel'] = TOKENLevel

result_seq = tagger(model, sent, get_seq = True)
list(zip(sent.sentence.split(' '), result_seq))

[('（', '标点-B'),
 ('6', '数字-B'),
 ('）', '标点-B'),
 ('不', '无-B'),
 ('能', '有-B'),
 ('规', '定性-B'),
 ('律', '定性-I'),
 ('进', '事件-B'),
 ('食', '事件-I'),
 ('，', '标点-B'),
 ('或', '连词-B'),
 ('不', '无-B'),
 ('能', '有-B'),
 ('配', '事件-B'),
 ('合', '事件-I'),
 ('饮', '身体功能-B'),
 ('食', '身体功能-I'),
 ('干', '定性-B'),
 ('预', '定性-I'),
 ('者', '代词-B')]

In [24]:
result_entities = tagger(model, sent)
sentence = sent.sentence.split(' ')

L = []
for i in result_entities:
    s, e, label = i
    L.append((''.join(sentence[s:e]), s, e, label))
L

[('（', 0, 1, '标点'),
 ('6', 1, 2, '数字'),
 ('）', 2, 3, '标点'),
 ('不', 3, 4, '无'),
 ('能', 4, 5, '有'),
 ('规律', 5, 7, '定性'),
 ('进食', 7, 9, '事件'),
 ('，', 9, 10, '标点'),
 ('或', 10, 11, '连词'),
 ('不', 11, 12, '无'),
 ('能', 12, 13, '有'),
 ('配合', 13, 15, '事件'),
 ('饮食', 15, 17, '身体功能'),
 ('干预', 17, 19, '定性'),
 ('者', 19, 20, '代词')]